In [188]:
import pandas as pd
import numpy as np

In [189]:
df1 = pd.read_csv("Plant_1_Generation_Data.csv")
df2 = pd.read_csv("Plant_1_Weather_Sensor_Data.csv")

In [190]:
df1['DATE_TIME'] = df1['DATE_TIME'].astype('datetime64[ns]')
df2['DATE_TIME'] = df2['DATE_TIME'].astype('datetime64[ns]').replace('/','-')

df = pd.merge(df1,df2, how='left', on='DATE_TIME')

In [191]:
df['DATE'] = df['DATE_TIME'].dt.date

df = df.drop(columns=['PLANT_ID_x', 'SOURCE_KEY_x','PLANT_ID_y', 'SOURCE_KEY_y'])

df.head()

,DATE_TIME,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,DATE
0,2020-05-15,0.0,0.0,0.0,6259559.0,25.184316,22.857507,0.0,2020-05-15
1,2020-05-15,0.0,0.0,0.0,6183645.0,25.184316,22.857507,0.0,2020-05-15
2,2020-05-15,0.0,0.0,0.0,6987759.0,25.184316,22.857507,0.0,2020-05-15
3,2020-05-15,0.0,0.0,0.0,7602960.0,25.184316,22.857507,0.0,2020-05-15
4,2020-05-15,0.0,0.0,0.0,7158964.0,25.184316,22.857507,0.0,2020-05-15


In [192]:
def filling_droping(x):
    x = x[x['IRRADIATION'] != 0.0]
    
    x = x.fillna(method = 'ffill')
    
    return x

In [271]:
df = filling_droping(df)

C:\Users\kkuri\AppData\Local\Temp\ipykernel_7564\4007533093.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  x = x.fillna(method = 'ffill')


In [272]:
def new_cols(x):
    daily_data = x.groupby('DATE')['DC_POWER'].mean()
    x['DAY_DC_POWER'] = x['DATE'].map(daily_data)
    
    weekly_sum = daily_data.rolling(window=7).sum()
    x['WEEK_DC_POWER'] = x['DATE'].map(weekly_sum)
    
    x['WEEK_DC_POWER'].fillna(0, inplace= True)
    
    return x

In [273]:
def time_cols(x) :
    x['Month'] = x['DATE_TIME'].dt.month
    x['Day'] = x['DATE_TIME'].dt.day

    x['Hour'] = x['DATE_TIME'].dt.hour
    x['Minute'] = x['DATE_TIME'].dt.minute

    x['Day_of_week'] = x['DATE_TIME'].dt.weekday
    
    
    return x

In [274]:
def cyclic_encoding_time(x):
    x_encoded = x.copy()
    
    x_encoded['hour_sin'] = np.sin(2 * np.pi * x_encoded['Hour'] / 24)
    x_encoded['hour_cos'] = np.cos(2 * np.pi * x_encoded['Hour'] / 24)

    x_encoded['minute_sin'] = np.sin(2 * np.pi * x_encoded['Minute'] / 60)
    x_encoded['minute_cos'] = np.cos(2 * np.pi * x_encoded['Minute'] / 60)
    
    return x_encoded.drop(columns=['Hour', 'Minute', ])

In [302]:
def cyclic_encoding_date(x) :
    x_encoded = x.copy()
    
    x_encoded['month_sin'] = np.sin(2 * np.pi * (x_encoded['Month'] -1) / 12)
    x_encoded['month_cos'] = np.cos(2 * np.pi * (x_encoded['Month'] -1) / 12)

    x_encoded['day_sin'] = np.sin(2 * np.pi * x_encoded['Day']- 1/ 31 )
    x_encoded['day_cos'] = np.cos(2 * np.pi * x_encoded['Day'] -1/ 31 )

    x_encoded['week_day_sin'] = np.sin(2 * np.pi * x_encoded['Day_of_week'] / 7)
    x_encoded['week_day_cos'] = np.cos(2 * np.pi * x_encoded['Day_of_week'] / 7)
    
    return x_encoded.drop(columns=['Month','Day','Day_of_week'])

In [303]:
from sklearn import set_config
set_config('diagram')
set_config(transform_output='pandas')

In [304]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [323]:
Adding_cols = FunctionTransformer(time_cols)
    


encoding_time = Pipeline([
    ('Time encoding', FunctionTransformer(cyclic_encoding_time)),
])

encoding_date = Pipeline([
    ('Date encoding', FunctionTransformer(cyclic_encoding_date)),
])


In [ ]:
cols_time = ['Hour','Minute','DATE_TIME','DATE']
cols_date = ['Month','Day','Day_of_week','DATE_TIME','DATE']

In [ ]:
preprocessing = ColumnTransformer([
    ('time encoding',encoding_time,cols_time),
    ('date encoding', encoding_date,cols_date),
],remainder='passthrough')

In [368]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

In [369]:
df = new_cols(df)
df.columns

Index(['DATE_TIME', 'DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD',
       'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION', 'DATE',
       'DAY_DC_POWER', 'WEEK_DC_POWER'],
      dtype='object')

In [370]:
x = df.drop(columns=['DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD','DAY_DC_POWER', 'WEEK_DC_POWER'])
y = df[['DC_POWER', 'WEEK_DC_POWER']]

In [371]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.33, random_state=42)

In [372]:
pipe = Pipeline([
    ('Adding cols', Adding_cols),
    ('Preprocessing', preprocessing),
    ('Dropping', FunctionTransformer(lambda x : x.drop(columns=['DATE_TIME', 'DATE']))),
    ('Scale', StandardScaler()),
    ('Clf', MultiOutputRegressor(RandomForestRegressor(n_estimators=100)))
])

In [373]:
pipe

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('Adding cols', ...), ('Preprocessing', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"func func: callable, default=NoneThe callable to use for the transformation. This will be passedthe same arguments as transform, with args and kwargs forwarded.If func is None, then func will be the identity function.",<function tim...001B446EAC460>
,"inverse_func inverse_func: callable, default=NoneThe callable to use for the inverse transformation. This will bepassed the same arguments as inverse transform, with args andkwargs forwarded. If inverse_func is None, then inverse_funcwill be the identity function.",None
,"validate validate: bool, default=FalseIndicate that the input X array should be checked before calling``func``. The possibilities are:- If False, there is no input validation.- If True, then X will be converted to a 2-dimensional NumPy array or sparse matrix. If the conversion is not possible an exception is raised... versionchanged:: 0.22 The default of ``validate`` changed from True to False.",False
,"accept_sparse accept_sparse: bool, default=FalseIndicate that func accepts a sparse matrix as input. If validate isFalse, this has no effect. Otherwise, if accept_sparse is false,sparse matrix inputs will cause an exception to be raised.",False
,"check_inverse check_inverse: bool, default=TrueWhether to check that or ``func`` followed by ``inverse_func`` leads tothe original inputs. It can be used for a sanity check, raising awarning when the condition is not fulfilled... versionadded:: 0.20",True
,"feature_names_out feature_names_out: callable, 'one-to-one' or None, default=NoneDetermines the list of feature names that will be returned by the`get_feature_names_out` method. If it is 'one-to-one', then the outputfeature names will be equal to the input feature names. If it is acallable, then it must take two positional arguments: this`FunctionTransformer` (`self`) and an array-like of input feature names(`input_features`). It must return an array-like of output featurenames. The `get_feature_names_out` method is only defined if`feature_names_out` is not None.See ``get_feature_names_out`` for more details... versionadded:: 1.1",None
,"kw_args kw_args: dict, default=NoneDictionary of additional keyword argume

In [374]:
pipe.fit(x_train,y_train)

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
cyclic_encoding_date(time_cols(x_train)[cols_date])

,DATE_TIME,DATE,month_sin,month_cos,day_sin,day_cos,week_day_sin,week_day_cos
7353,2020-05-18 15:15:00,2020-05-18,0.866025,-5.000000e-01,-0.032252,0.99948,0.000000,1.000000
53622,2020-10-06 19:15:00,2020-10-06,-1.000000,-1.836970e-16,-0.032252,0.99948,0.781831,0.623490
18487,2020-05-24 15:15:00,2020-05-24,0.866025,-5.000000e-01,-0.032252,0.99948,-0.781831,0.623490
30511,2020-05-30 18:15:00,2020-05-30,0.866025,-5.000000e-01,-0.032252,0.99948,-0.974928,-0.222521
24439,2020-05-27 12:00:00,2020-05-27,0.866025,-5.000000e-01,-0.032252,0.99948,0.974928,-0.222521
...,...,...,...,...,...,...,...,...
20203,2020-05-25 11:30:00,2020-05-25,0.866025,-5.000000e-01,-0.032252,0.99948,0.000000,1.000000
61442,2020-06-14 12:00:00,2020-06-14,0.500000,-8.660254e-01,-0.032252,0.99948,-0.781831,0.623490
53416,2020-10-06 16:45:00,2020-10-06,-1.000000,-1.836970e-16,-0.032252,0.99948,0.781831,0.623490
1349,2020-05-15 15:30:00,2020-05-15,0.866025,-5.000000e-01,-0.032252,0.99948,-0.433884,-0.900969


In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48790 entries, 489 to 68337
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DATE_TIME            48790 non-null  datetime64[ns]
 1   AMBIENT_TEMPERATURE  48790 non-null  float64       
 2   MODULE_TEMPERATURE   48790 non-null  float64       
 3   IRRADIATION          48790 non-null  float64       
 4   DATE                 48790 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 2.2+ MB
